Start by importing all relevant modules

In [19]:
# Importing necessary libraries
# Basics
import pandas as pd
import numpy as np
import itertools
from io import StringIO
from datetime import datetime, timedelta

# Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.express as px
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.pylab import rcParams
import time

# Modeling libraries
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA        
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import TimeSeriesSplit 
from pmdarima import auto_arima      

from prophet import Prophet 

#Model deployment libraries
import joblib    


# Warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.filterwarnings('ignore')

# Custom Options for displaying rows.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',100)

Reading sample data from TfNSW:

In [20]:
df = pd.read_json("data/NSW response - 2022-03-13.json")
df.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,211420,700405713,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:08:33,14,West Ryde Car Park,211420TPR001
1,211420,700406314,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:18:34,14,West Ryde Car Park,211420TPR001
2,211420,700406916,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:28:36,14,West Ryde Car Park,211420TPR001
3,211420,700407517,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:38:37,14,West Ryde Car Park,211420TPR001
4,211420,700408119,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:48:39,14,West Ryde Car Park,211420TPR001


Looking at dataframe columns:

In [21]:
df.columns

Index(['tsn', 'time', 'spots', 'zones', 'ParkID', 'occupancy', 'MessageDate',
       'facility_id', 'facility_name', 'tfnsw_facility_id'],
      dtype='object')

And dataframe info

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tsn                191 non-null    int64 
 1   time               191 non-null    int64 
 2   spots              191 non-null    int64 
 3   zones              191 non-null    object
 4   ParkID             191 non-null    int64 
 5   occupancy          191 non-null    object
 6   MessageDate        191 non-null    object
 7   facility_id        191 non-null    int64 
 8   facility_name      191 non-null    object
 9   tfnsw_facility_id  191 non-null    object
dtypes: int64(5), object(5)
memory usage: 15.1+ KB


The values in the time column are given in seconds since year 2000. Converting it to more meaningful data that can be interpreted:

In [24]:
def extract_date_time(message_date):
  date = message_date.split('T')[0]
  time = message_date.split('T')[1]
  return date,time

In [26]:
df_copy = df.copy()

df_copy[['date','time']] = df['MessageDate'].apply(extract_date_time).apply(pd.Series)
df_copy.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id,date
0,211420,00:08:33,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:08:33,14,West Ryde Car Park,211420TPR001,2022-03-13
1,211420,00:18:34,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:18:34,14,West Ryde Car Park,211420TPR001,2022-03-13
2,211420,00:28:36,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:28:36,14,West Ryde Car Park,211420TPR001,2022-03-13
3,211420,00:38:37,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:38:37,14,West Ryde Car Park,211420TPR001,2022-03-13
4,211420,00:48:39,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:48:39,14,West Ryde Car Park,211420TPR001,2022-03-13


Creating column with day of the week

In [27]:
df_copy['day_of_week'] = pd.to_datetime(df_copy['date']).apply(lambda x: x.strftime('%A'))
df_copy.head()

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id,date,day_of_week
0,211420,00:08:33,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:08:33,14,West Ryde Car Park,211420TPR001,2022-03-13,Sunday
1,211420,00:18:34,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:18:34,14,West Ryde Car Park,211420TPR001,2022-03-13,Sunday
2,211420,00:28:36,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:28:36,14,West Ryde Car Park,211420TPR001,2022-03-13,Sunday
3,211420,00:38:37,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:38:37,14,West Ryde Car Park,211420TPR001,2022-03-13,Sunday
4,211420,00:48:39,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:48:39,14,West Ryde Car Park,211420TPR001,2022-03-13,Sunday


Reordering columns:

In [28]:
df_copy = df_copy[['tsn','day_of_week','date','time','spots','zones','ParkID','occupancy','MessageDate','facility_id','facility_name','tfnsw_facility_id']]
df_copy.head()

,tsn,day_of_week,date,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,211420,Sunday,2022-03-13,00:08:33,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:08:33,14,West Ryde Car Park,211420TPR001
1,211420,Sunday,2022-03-13,00:18:34,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:18:34,14,West Ryde Car Park,211420TPR001
2,211420,Sunday,2022-03-13,00:28:36,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:28:36,14,West Ryde Car Park,211420TPR001
3,211420,Sunday,2022-03-13,00:38:37,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:38:37,14,West Ryde Car Park,211420TPR001
4,211420,Sunday,2022-03-13,00:48:39,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '28302', 'total': '2', 'monthlies': N...",2022-03-13T00:48:39,14,West Ryde Car Park,211420TPR001


Dropping columns not needed now:

In [29]:
df_copy.drop(['tfnsw_facility_id', 'ParkID','MessageDate', 'facility_id'], axis=1, inplace=True)
df_copy.head()

,tsn,day_of_week,date,time,spots,zones,occupancy,facility_name
0,211420,Sunday,2022-03-13,00:08:33,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...","{'loop': '28302', 'total': '2', 'monthlies': N...",West Ryde Car Park
1,211420,Sunday,2022-03-13,00:18:34,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...","{'loop': '28302', 'total': '2', 'monthlies': N...",West Ryde Car Park
2,211420,Sunday,2022-03-13,00:28:36,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...","{'loop': '28302', 'total': '2', 'monthlies': N...",West Ryde Car Park
3,211420,Sunday,2022-03-13,00:38:37,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...","{'loop': '28302', 'total': '2', 'monthlies': N...",West Ryde Car Park
4,211420,Sunday,2022-03-13,00:48:39,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...","{'loop': '28302', 'total': '2', 'monthlies': N...",West Ryde Car Park


Converting the zones column to its own dataframe:

In [49]:
df_zones = pd.DataFrame(columns=['spots','zone_id','zone_name','parent_zone_id','occupancy'])
rename_format = {
    0:'spots',
    1:'zone_id',
    2:'zone_name',
    3:'parent_zone_id',
    4:'occupancy loops',
    5:'occupancy total',
    6:'occupancy monthlies',
    7:'occupancy open_gate',
    8:'occupancy transients'}



for key,value in df['zones'].items():    
    # Normalize values in each record in zones column
    val = pd.json_normalize(value)
    
    # Convert it to a dataframe
    temp_holder = pd.DataFrame.from_dict(val.values)
    # Renaming columns
    temp_holder.rename(mapper=rename_format, axis=1, inplace=True)
    # merge it with main dataframe
    df_zones = pd.concat([df_zones, temp_holder], ignore_index=True)

df_zones.head()

,spots,zone_id,zone_name,parent_zone_id,occupancy,occupancy loops,occupancy total,occupancy monthlies,occupancy open_gate,occupancy transients
0,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
1,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
2,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
3,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
4,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None


In [50]:
df_occupancy = pd.DataFrame(
    columns=['spots', 'zone_id', 'zone_name', 'parent_zone_id', 'occupancy'])
rename_format = {
    0: 'spots',
    1: 'zone_id',
    2: 'zone_name',
    3: 'parent_zone_id',
    4: 'occupancy loops',
    5: 'occupancy total',
    6: 'occupancy monthlies',
    7: 'occupancy open_gate',
    8: 'occupancy transients'}


for key, value in df['zones'].items():
    # Normalize values in each record in zones column
    val = pd.json_normalize(value)

    # Convert it to a dataframe
    temp_holder = pd.DataFrame.from_dict(val.values)
    # Renaming columns
    temp_holder.rename(mapper=rename_format, axis=1, inplace=True)
    # merge it with main dataframe
    df_occupancy = pd.concat([df_occupancy, temp_holder], ignore_index=True)

df_occupancy.head()

,spots,zone_id,zone_name,parent_zone_id,occupancy,occupancy loops,occupancy total,occupancy monthlies,occupancy open_gate,occupancy transients
0,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
1,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
2,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
3,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
4,151,1,SYD372 West Ryde Park&Ride,0,NaN,28302,2,None,None,None
